#06_SparkDataAnal.ipynb
TLC Trip Record Data
출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [ ]:
#2015-summary.json

In [2]:
df = spark.read.format('json')\
        .load("learning_spark_data/2015-summary.json")

In [3]:
df.count()

256

In [4]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

In [5]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
df.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sint Maarten', count=325),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Marshall Islands', count=39),
 

In [7]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [8]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [10]:
#도착국가명 중복제거
df.select('DEST_COUNTRY_NAME').distinct().show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [13]:
df1 = df.select('DEST_COUNTRY_NAME').distinct().cache()
df1.count()

132

In [14]:
# ROW class를 이용한 단일 레코드 생성

from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [15]:
# 새로운 컬럼 추가하기
from pyspark.sql.functions import expr

df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME')) #expr sql표현식을 받아 생성
df3

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, withinCountry: boolean]

In [16]:
df3.show(3)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
+-----------------+-------------------+-----+-------------+
only showing top 3 rows



In [18]:
df3.filter(expr('withinCountry==True')).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [ ]:
#case when 카운트 10이하 under, 이상 upper로 변환 > category 컬럼 추가

In [19]:
df4 = df.withColumn('category', expr("CASE WHEN count<10 THEN 'under' WHEN count>=10 THEN 'upper' END"))
df4.show(10)

+-----------------+-------------------+-----+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|category|
+-----------------+-------------------+-----+--------+
|    United States|            Romania|   15|   upper|
|    United States|            Croatia|    1|   under|
|    United States|            Ireland|  344|   upper|
|            Egypt|      United States|   15|   upper|
|    United States|              India|   62|   upper|
|    United States|          Singapore|    1|   under|
|    United States|            Grenada|   62|   upper|
|       Costa Rica|      United States|  588|   upper|
|          Senegal|      United States|   40|   upper|
|          Moldova|      United States|    1|   under|
+-----------------+-------------------+-----+--------+
only showing top 10 rows



In [ ]:
#DataFrame의 select(), where(), filter() 트랜스포메이션
#show(), count() 액션

In [20]:
spark.stop()

In [ ]:
#emp_df, dept_df

In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SecondSparkSessionApp").getOrCreate()

In [23]:
emp_df = spark.read.format('csv')\
        .option('header','true')\
        .option('inferSchema', 'true')\
        .load("learning_spark_data/emp.csv")      

emp_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [25]:
dept_df = spark.read.format('csv')\
        .option('header','true')\
        .option('inferSchema', 'true')\
        .load("learning_spark_data/dept.csv")      

dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [26]:
#컬럼명은 대소문자 구분하지 않는다.
emp_df.select('ENAME', 'DEPTNO').show()

+------+------+
| ENAME|DEPTNO|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [27]:
#filter()랑 동일함.
emp_df.select('*').where('deptno=20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [29]:
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [30]:
from pyspark.sql.functions import countDistinct

emp_df.select( countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [31]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select( approx_count_distinct('job', 0.1 )).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [44]:
from pyspark.sql.functions import min, max, count, first, last, sum, avg, mean

In [35]:
# first,last,min,max,sum, avg -> (expr : sql 문장 x ), function으로 처리
emp_df.select(  count('sal') ).show()

+----------+
|count(sal)|
+----------+
|        15|
+----------+



In [37]:
emp_df.select(count('empno'), count('*'), count('comm'), max('ename'), max('comm')).show()

+------------+--------+-----------+----------+---------+
|count(empno)|count(1)|count(comm)|max(ename)|max(comm)|
+------------+--------+-----------+----------+---------+
|          15|      15|          4|      WARD|     1400|
+------------+--------+-----------+----------+---------+



In [51]:
emp_df.select(sum('sal'), mean('sal'), avg('sal')).show()

+--------+------------------+------------------+
|sum(sal)|          avg(sal)|          avg(sal)|
+--------+------------------+------------------+
|   32225|2148.3333333333335|2148.3333333333335|
+--------+------------------+------------------+



In [49]:
from pyspark.sql.functions import col
emp_df.select(sum(col('sal'))).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [52]:
emp_df.selectExpr('    sum(distinct sal) ').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [ ]:
#total_salary / total_transaction, avg_salary, mean_salary

In [53]:
emp_df.select(
    count('sal').alias('total_transaction'),
    sum('sal').alias('total_salary'),
    avg('sal').alias('avg_salary'),
    mean('sal').alias('mean_salary')
).selectExpr(
    'total_salary/total_transaction',
    'avg_salary',
    'mean_salary'
).show()

+----------------------------------+------------------+------------------+
|(total_salary / total_transaction)|        avg_salary|       mean_salary|
+----------------------------------+------------------+------------------+
|                2148.3333333333335|2148.3333333333335|2148.3333333333335|
+----------------------------------+------------------+------------------+



In [54]:
# 그룹화
emp_df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [56]:
# select job, 
#     count(job),
#     sum(sal)
# group by job
group_df = emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal')
)
group_df.show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [ ]:
#sal의 평균 SAL_AVG, 표준편차 SAL_STDEV 를 job별로 계산해서 출력, 소수점2자리

In [57]:
from pyspark.sql.functions import stddev, round
emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    round(avg('sal'), 2).alias('SAL_AVG'),
    round(stddev('sal'), 2).alias('SAL_STDEV')
).show()

+---------+---+-------+---------+
|      job|qty|SAL_AVG|SAL_STDEV|
+---------+---+-------+---------+
|  ANALYST|  2| 3000.0|      0.0|
| SALESMAN|  4| 1400.0|   177.95|
|    CLERK|  5| 1470.0|   984.63|
|  MANAGER|  3|2758.33|   274.24|
|PRESIDENT|  1| 5000.0|     NULL|
+---------+---+-------+---------+



In [ ]:
#급여 TOP10 구하기

In [58]:
emp_df.orderBy(emp_df.sal.desc()).limit(10).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+



In [60]:
#윈도우 함수
from pyspark.sql.functions import desc, rank
from pyspark.sql.window import Window
windowspec = Window.orderBy(desc('sal'))
salAllRank = rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [64]:
#emp_df.show(10)

In [63]:
emp_df.withColumn('salary_rank', salAllRank).show(3)

+-----+-----+---------+----+----------+----+----+------+-----------+
|empno|ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+-----+---------+----+----------+----+----+------+-----------+
| 7839| KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292| JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788|SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
+-----+-----+---------+----+----------+----+----+------+-----------+
only showing top 3 rows



In [ ]:
#직무별로 rank 작성
#Window.partitionBy()
#job_rank_df 작성

In [65]:
emp_df.dtypes

[('empno', 'int'),
 ('ename', 'string'),
 ('job', 'string'),
 ('mgr', 'int'),
 ('hiredate', 'date'),
 ('sal', 'int'),
 ('comm', 'int'),
 ('deptno', 'int')]

In [66]:
windowspec1 = Window.partitionBy('job').orderBy(desc('sal'))
salJobRank = rank().over(windowspec1)
salJobRank

Column<'RANK() OVER (PARTITION BY job ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [67]:
emp_df.withColumn('salary_job_rank', salJobRank).show(3)

+-----+-----+-------+----+----------+----+----+------+---------------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|salary_job_rank|
+-----+-----+-------+----+----------+----+----+------+---------------+
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|              1|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|              1|
| 9292| JACK|  CLERK|7782|1982-01-23|3200|NULL|    70|              1|
+-----+-----+-------+----+----------+----+----+------+---------------+
only showing top 3 rows



In [ ]:
# 부서별 순위

In [69]:
from pyspark.sql.functions import rank, dense_rank, row_number, col, desc

# 2. 부서별 급여 순위
dept_window_spec = Window.partitionBy('deptno').orderBy(desc('sal'))
df_with_dept_rank = emp_df.withColumn('dept_salary_rank', 
                                  row_number().over(dept_window_spec))
df_with_dept_rank.select('ename', 'deptno', 'sal', 'dept_salary_rank').show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               2|
| JONES|    20|2975|               3|
| ADAMS|    20|1100|               4|
| SMITH|    20| 800|               5|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               5|
| JAMES|    30| 950|               6|
|  JACK|    70|3200|               1|
+------+------+----+----------------+



In [ ]:
#부서별 누적 급여 sum('sal').over()

In [70]:
window_spec_sum = Window.partitionBy('deptno').orderBy('empno')
df_cumulative = emp_df.withColumn('cumulative_salary', 
                                sum('sal').over(window_spec_sum))
df_cumulative.select('ename', 'deptno', 'sal', 'cumulative_salary').show()

+------+------+----+-----------------+
| ename|deptno| sal|cumulative_salary|
+------+------+----+-----------------+
| CLARK|    10|2450|             2450|
|  KING|    10|5000|             7450|
|MILLER|    10|1300|             8750|
| SMITH|    20| 800|              800|
| JONES|    20|2975|             3775|
| SCOTT|    20|3000|             6775|
| ADAMS|    20|1100|             7875|
|  FORD|    20|3000|            10875|
| ALLEN|    30|1600|             1600|
|  WARD|    30|1250|             2850|
|MARTIN|    30|1250|             4100|
| BLAKE|    30|2850|             6950|
|TURNER|    30|1500|             8450|
| JAMES|    30| 950|             9400|
|  JACK|    70|3200|             3200|
+------+------+----+-----------------+



In [ ]:
#부서별 평균급여와 직원 개별 급여 비교

In [72]:
window_spec_avg = Window.partitionBy('deptno')
df_avg_compare = emp_df.withColumn('dept_avg_salary', 
                        avg('sal').over(window_spec_avg))
df_avg_compare.select('ename', 'deptno', 'sal', 'dept_avg_salary').show()

+------+------+----+------------------+
| ename|deptno| sal|   dept_avg_salary|
+------+------+----+------------------+
| CLARK|    10|2450|2916.6666666666665|
|  KING|    10|5000|2916.6666666666665|
|MILLER|    10|1300|2916.6666666666665|
| SMITH|    20| 800|            2175.0|
| JONES|    20|2975|            2175.0|
| SCOTT|    20|3000|            2175.0|
| ADAMS|    20|1100|            2175.0|
|  FORD|    20|3000|            2175.0|
| ALLEN|    30|1600|1566.6666666666667|
|  WARD|    30|1250|1566.6666666666667|
|MARTIN|    30|1250|1566.6666666666667|
| BLAKE|    30|2850|1566.6666666666667|
|TURNER|    30|1500|1566.6666666666667|
| JAMES|    30| 950|1566.6666666666667|
|  JACK|    70|3200|            3200.0|
+------+------+----+------------------+



In [ ]:
# 부서별 직업별 소계

In [ ]:
# 직원, 부서 조인